# Mermaid charting 
First we need to load support in the extension lab add on. 

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*

## Magic command
Now we have a magic command for mermaid

In [ ]:
#!mermaid
sequenceDiagram
    participant Alice
    participant Bob
    Alice->>John: Hello John, how are you?
    loop Healthcheck
        John->>John: Fight against hypochondria
    end
    Note right of John: Rational thoughts 

## PowerShell 
Define a PowerShell function `Out Mermaid` (with an alias of `Mermaid` so we can call it in more dsl-y style);   
it accepts a mermaid chart definiton as a parameter (example with the definition) or from the   
pipeline (see subsequent example) and  this outputs the minimum correct HTML / Javascript but   
**depends on the kernel extension being loaded** 

Test with examples from [the mermaid home page](https://mermaid-js.github.io/mermaid/#/) -    



In [ ]:
function Out-Mermaid {
    [alias('Mermaid')]
    param (
        [parameter(ValueFromPipeline=$true,Mandatory=$true,Position=0)]
        $Text
    )
    begin {
        $mermaid = ""
        $html    =  @"
        <script type="text/javascript">
        configureRequireFromExtension('Mermaid','1.0.0')(['Mermaid/mermaidapi'], (mermaid) => {
            let renderTarget = document.getElementById('~~guid~~');
            mermaid.render('mermaid_~~guid~~',   ``~~Mermaid~~ ``,  g => { renderTarget.innerHTML = g }); 
        }, (error) => { console.log(error);  });
        </script><div  style="background-color:white"; id="~~guid~~"></div>'     
"@  }
    process {
        $Mermaid +=  ("`r`n" + $Text -replace '^[\r\n]+','' -replace '[\r\n]+$','') 
    }
    end     {
        $GUID    =  ([guid]::NewGuid().ToString() -replace '\W','') 
        [Microsoft.DotNet.Interactive.Kernel]::HTML(($html -replace  '~~Mermaid~~',$mermaid -replace '~~guid~~',$GUID )) | Out-Display -MimeType "text/html"
    }
}

Mermaid @'
sequenceDiagram
    participant Alice
    participant Bob
    Alice->>John: Hello John, how are you?
    loop Healthcheck
        John->>John: Fight against hypochondria
    end
    Note right of John: Rational thoughts
'@

In [ ]:
@'
graph TD;
    A-->B;
    A-->C;
    B-->D;
    C-->D;
'@ | Out-Mermaid

In [ ]:
Out-Mermaid @'
gantt
dateFormat  YYYY-MM-DD
title Adding GANTT diagram to mermaid
excludes weekdays 2014-01-10

section A section
Completed task            :done,    des1, 2014-01-06,2014-01-08
Active task               :active,  des2, 2014-01-09, 3d
Future task               :         des3, after des2, 5d
Future task2               :         des4, after des3, 5d
'@

In [ ]:
Mermaid @'
classDiagram
Class01 <|-- AveryLongClass : Cool
Class03 *-- Class04
Class05 o-- Class06
Class07 .. Class08
Class09 --> C2 : Where am i?
Class09 --* C3
Class09 --|> Class07
Class07 : equals()
Class07 : Object[] elementData
Class01 : size()
Class01 : int chimp
Class01 : int gorilla
Class08 <--> C2: Cool label
'@

In [ ]:
Mermaid @'
erDiagram
    CUSTOMER ||--o{ ORDER : places
    ORDER ||--|{ LINE-ITEM : contains
    CUSTOMER }|..|{ DELIVERY-ADDRESS : uses
'@


In [ ]:
Mermaid @' 
journey
    title My working day
    section Go to work
      Make tea: 5: Me
      Go upstairs: 3: Me
      Do work: 1: Me, Cat
    section Go home
      Go downstairs: 5: Me
      Sit down: 5: Me
'@